In [8]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

device = torch.device("cuda")
print(os.getcwd())

c:\Users\hunte\coding projects


In [9]:
import torch
from transformers import BertModel, BertForMaskedLM, BertTokenizer

#checkpoint = torch.load('C:\\Users\\hunte\\coding projects\\ipynbs to get good text data\\example.pt')
model_path = 'C:/Users/hunte/coding projects/ipynbs to get good text data/example_fixed.pt'
# model_dict = torch.load(model_path)
# print(model_dict)
# print((model_dict.keys()))
# model = model_dict['model']
# # Set the model to evaluation mode
# model.eval()
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch = checkpoint['epoch']
#loss = checkpoint['loss']


In [10]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [11]:
ntokens = 65292  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

state_dict = torch.load(model_path)
model.load_state_dict(state_dict)

print(type(state_dict))
print(type(model))

for key, value in state_dict.items():
    print(key, value)




<class 'collections.OrderedDict'>
<class '__main__.TransformerModel'>
pos_encoder.pe tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
           0.0000e+00,  1.0000e+00]],

        [[ 8.4147e-01,  5.4030e-01,  7.9074e-01,  ...,  1.0000e+00,
           1.0965e-04,  1.0000e+00]],

        [[ 9.0930e-01, -4.1615e-01,  9.6811e-01,  ...,  1.0000e+00,
           2.1930e-04,  1.0000e+00]],

        ...,

        [[ 9.5625e-01, -2.9254e-01,  9.0551e-01,  ...,  8.2490e-01,
           5.2090e-01,  8.5362e-01]],

        [[ 2.7050e-01, -9.6272e-01,  2.1917e-01,  ...,  8.2483e-01,
           5.2100e-01,  8.5356e-01]],

        [[-6.6395e-01, -7.4778e-01, -6.3742e-01,  ...,  8.2476e-01,
           5.2109e-01,  8.5350e-01]]], device='cuda:0')
transformer_encoder.layers.0.self_attn.in_proj_weight tensor([[ 0.0493,  0.0749,  0.0089,  ..., -0.1834, -0.0171, -0.0850],
        [ 0.0855, -0.1409, -0.0879,  ..., -0.1463,  0.0855, -0.2376],
        [-0.1764, -0.0833, -0.0917,  ..., -0.000

In [12]:
from torchtext.data.utils import get_tokenizer
import numpy as np
import pickle

file = open("vocab.obj",'rb')
vocab = pickle.load(file)
file.close()
input_text = "This is a test sentence ."
print(type(vocab))

<class 'torchtext.vocab.vocab.Vocab'>


In [13]:
for x in range(0,300):
    print(vocab.lookup_token(x))
tokenizer = get_tokenizer('basic_english')
tokenized_text = tokenizer(input_text)
input_ids = torch.tensor(vocab(tokenized_text), dtype=torch.long)
print(input_ids)

<unk>
=
type
language
.
invalid
/
-
,
multiverseid
format
legality
legal
text
card
name
the
flavor
{
}
com
?
wizards
&
http
gatherer
image
ashx
handlers
a
imageurl
de
1
creature
you
—
self
of
to
'
+
it
)
(
t
date
2
la
that
if
’
s
hand
and
or
in
your
life
criatura
power
number
is
que
un
toughness
ability
commander
set
as
source
colors
3
this
r
u
names
subtypes
artist
supertypes
variations
timeshifted
on
starter
watermark
rulings
border
cmc
color_identity
foreign_names
id
image_url
layout
legalities
loyality
mana_cost
multiverse_id
original_text
original_type
printings
rarity
release_date
reserved
set_name
normal
w
target
g
b
duel
with
vintage
legacy
battlefield
an
spell
can
control
for
mana
turn
di
o
player
le
its
chinese
una
die
du
no
créature
from
be
cast
modern
creatura
any
put
when
kreatur
cards
japanese
cost
french
german
damage
italian
spanish
penny
do
will
4
del
x
may
el
à
brazil
portuguese
each
e
one
der
il
creatures
se
paupercommander
eine
des
turno
at
vous
have
um
en
0
simplif

In [14]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.encoder = model.encoder
        self.fc1 = torch.nn.Linear(768, 128)
        self.fc2 = torch.nn.Linear(128, 2)
        
    def forward(self, input_ids, attention_mask):
        # encode the input using the transformer model
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state[:, 0, :]
        
        # pass the embeddings through your neural network layers
        x = self.fc1(embeddings)
        x = torch.nn.functional.relu(x)
        x = self.fc2(x)
        
        return x

# initialize your neural network
net = MyNet()

# preprocess your input data
attention_mask = torch.ones(np.array(input_ids).shape)

# get the output of your neural network
outputs = net(input_ids, attention_mask)

<unk>
=
type
language
.
invalid
/
-
,
multiverseid
format
legality
legal
text
card
name
the
flavor
{
}
com
?
wizards
&
http
gatherer
image
ashx
handlers
a
imageurl
de
1
creature
you
—
self
of
to
'
+
it
)
(
t
date
2
la
that
if
’
s
hand
and
or
in
your
life
criatura
power
number
is
que
un
toughness
ability
commander
set
as
source
colors
3
this
r
u
names
subtypes
artist
supertypes
variations
timeshifted
on
starter
watermark
rulings
border
cmc
color_identity
foreign_names
id
image_url
layout
legalities
loyality
mana_cost
multiverse_id
original_text
original_type
printings
rarity
release_date
reserved
set_name
normal
w
target
g
b
duel
with
vintage
legacy
battlefield
an
spell
can
control
for
mana
turn
di
o
player
le
its
chinese
una
die
du
no
créature
from
be
cast
modern
creatura
any
put
when
kreatur
cards
japanese
cost
french
german
damage
italian
spanish
penny
do
will
4
del
x
may
el
à
brazil
portuguese
each
e
one
der
il
creatures
se
paupercommander
eine
des
turno
at
vous
have
um
en
0
simplif

TypeError: forward() got an unexpected keyword argument 'input_ids'